* Course    : Artificial Intelligence (CS6613) : New York University
* Project 1 : Surface Type Classification
* Author    : Harsh Thakkar (ht1215)

***Problem statement*** : Given the sensor measurement in the form of orientation values, angular velocity and linear acceleration values on different surfaces. Train machine learning model to predict the surface for separately given test data.

***Overview of the data*** : 
Training data (X) contains 3790 measurements each having 128 different sample reading and (y) contains 3790 rows denoting Surface type for the given series of measurements

Testing data contains 20 different measurements each having 128 sample reainds and (y) contains 20 rows denoting Surface Type

Pandas library is used to read data from csv files and store in the variables which will later be used for training and testing purposes

In [ ]:
import pandas as pd

x_train = pd.read_csv('/kaggle/input/career-con-2019/X_train.csv')
y_train = pd.read_csv('/kaggle/input/career-con-2019/y_train.csv')

x_new_train = pd.read_csv('/kaggle/input/new-dataset/X_train_new.csv')
y_new_train = pd.read_csv('/kaggle/input/new-dataset/y_train_new.csv')

x_test = pd.read_csv('/kaggle/input/new-dataset/X_test_new.csv')
y_test = pd.read_csv('/kaggle/input/new-dataset/y_test_new.csv')

In [ ]:
x_new_train.shape, x_test.shape, y_new_train.shape, y_test.shape

In the given dataset, we have total 9 different Surface types on which the robot has collected sensor data and that is stored in string format. To be used the data in machine learning model, it is computationally more convenient to convert the string format into corresponding integer format which is easy to compute and to convert back to string format when the operation is performed. Label binarizer and label encoder can be used to convert labels into integers. Label binarizer creates seperate copy for each input and stores as binary value where as label encoder classifies into number of different labels available. 

Here, y_new_train['surface'] column is labelised using label binariser and it can be seen that it stores binary values in matrix format where single row has one entry as 1 denoting the presence of that class. Numpy library is used to convert it into numpy array.

In [ ]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
import numpy as np
lb = LabelBinarizer()
le = LabelEncoder()

new_target = lb.fit_transform(y_new_train['surface'])
new_target = np.array(new_target)

In [ ]:
new_target

**DATA VISUALIZATION and DATA DEPENDANCY**
Seaborn library is very useful visualizing library. Countplot is used to plot number of different samples for each surface type. For example, there are 792 sample measurements for concrete surface, 310 samples for hard tiles large space surface. 

Heatmap helps in visualizing different dependancies between sensor values and gives idea about how they are proportional to each other.

In [ ]:
import seaborn as sns
from seaborn import countplot
import matplotlib.pyplot as plt
countplot(y='surface', data = y_train)
plt.show()

f, ax = plt.subplots(figsize= (8,8))
sns.heatmap(x_train.iloc[:,3:].corr(), annot = True, linewidth = .5, fmt = '.1f', ax = ax)

This section of code represents different sensor values concentrations for different surface types. This helps in differentiating and analysing what sensor values will be useful features for differentiating between different surface types

In [ ]:
def plot_feature_class_distribution(classes,tt, features,a=5,b=2):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(a,b,figsize=(22,36))

    for feature in features:
        i += 1
        plt.subplot(a,b,i)
        for clas in classes:
            ttc = tt[tt['surface']==clas]
            sns.kdeplot(ttc[feature], bw=0.5,label=clas)
        plt.xlabel(feature, fontsize=9)
        locs, labels = plt.xticks()
        plt.tick_params(axis='x', which='major', labelsize=8)
        plt.tick_params(axis='y', which='major', labelsize=8)
    plt.show();

features = x_train.columns.values[3:]
classes = (y_train['surface'].value_counts()).index
aux = x_train.merge(y_train, on='series_id', how='inner')
plot_feature_class_distribution(classes, aux, features, a = int (features.shape[0]/2))

Gimbal lock : Euler measurement can create gimbal lock when 2 of the axis aling in the same plane 
and in this situation rotating those 2 axis in the same plane can no longer move the object in any
other direction. That is why we are given data in quaternion formation and we can convert quaternion to euler.

Reference : [wiki](https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles)

[youtube](https://www.youtube.com/watch?v=zc8b2Jo7mno)

In [ ]:
def quaternion_to_euler(x, y, z, w):
    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.atan2(t0, t1)
    
    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.atan2(t3, t4)

    return X, Y, Z

def feature_extraction (data):
    data['norm_orientation'] = ((data['orientation_X']**2 + data['orientation_Y']**2 + data['orientation_Z']**2 + data['orientation_W']**2)**0.5)
    data['norm_X'] = data['orientation_X'] / data['norm_orientation']
    data['norm_Y'] = data['orientation_Y'] / data['norm_orientation']
    data['norm_Z'] = data['orientation_Z'] / data['norm_orientation']
    data['norm_W'] = data['orientation_W'] / data['norm_orientation'] 
    data['ang_y_vs_z'] = data['angular_velocity_Y'] * data['angular_velocity_Z']
    data['norm_angular'] = ((data['angular_velocity_X']**2 + data['angular_velocity_Y']**2 + data['angular_velocity_Z']**2)**0.5)
    data['norm_linear'] = ((data['linear_acceleration_X']**2 + data['linear_acceleration_Y']**2 + data['linear_acceleration_Z']**2)**0.5)
    data['acc_vs_velocity'] = data['norm_angular'] / data['norm_linear']
    x, y, z, w = data['norm_X'].tolist(), data['norm_Y'].tolist(), data['norm_Z'].tolist(), data['norm_W'].tolist()
    nx, ny, nz = [], [], []
    for i in range(len(x)):
        xx, yy, zz = quaternion_to_euler(x[i], y[i], z[i], w[i])
        nx.append(xx)
        ny.append(yy)
        nz.append(zz)
    data['euler_x'] = nx
    data['euler_y'] = ny
    data['euler_z'] = nz
    return data

x_train = feature_extraction(x_train)
x_new_train = feature_extraction(x_new_train)
x_test = feature_extraction(x_test)

In [ ]:
features = x_new_train.columns.values[4:]
classes = (y_new_train['surface'].value_counts()).index
aux = x_new_train.merge(y_new_train, on='series_id', how='inner')
plot_feature_class_distribution(classes, aux, features, a = int (features.shape[0]/2))

By looking at the graphs above, Some of the columns actually show variation for different surface types
and those features can be used to train the model. Below are the new data frame with the only important 
features extracting from above data frame

In [ ]:
x_train = x_train[['series_id', 'norm_X', 'norm_Y', 'norm_Z', 'norm_W','linear_acceleration_X', 'linear_acceleration_Y', 'linear_acceleration_Z', 'angular_velocity_X', 'angular_velocity_Y', 'angular_velocity_Z']]
x_new_train = x_new_train[['norm_X', 'norm_Y', 'norm_Z', 'norm_W','linear_acceleration_X', 'linear_acceleration_Y', 'linear_acceleration_Z', 'angular_velocity_X', 'angular_velocity_Y', 'angular_velocity_Z']]
x_test = x_test[['norm_X', 'norm_Y', 'norm_Z', 'norm_W','linear_acceleration_X', 'linear_acceleration_Y', 'linear_acceleration_Z', 'angular_velocity_X', 'angular_velocity_Y', 'angular_velocity_Z']]

As there are 128 samples for different measurements, this function combines and reshapes the resultant array in 3 dimensional array with (samples, measurement count, total features)

Training sample then becomes of the dimension (3790, 128, 10) and testing sample becomes of the dimension of (20, 128, 10)

In [ ]:
def combine_feature(train):
    segments = []

    for i in range(0, len(train), 128): 
        ox = train.norm_X.values[i:i+128]
        oy = train.norm_Y.values[i:i+128]
        oz = train.norm_Z.values[i:i+128]
        ow = train.norm_W.values[i:i+128]
        ax = train.angular_velocity_X.values[i:i+128]
        ay = train.angular_velocity_Y.values[i:i+128]
        az = train.angular_velocity_Z.values[i:i+128]
        lx = train.linear_acceleration_X.values[i:i+128]
        ly = train.linear_acceleration_Y.values[i:i+128]
        lz = train.linear_acceleration_Z.values[i:i+128]
        
        segments.append([ox, oy, oz, ow, ax, ay,az,lx, ly, lz])
    segments = np.asarray(segments, dtype= np.float32).reshape(-1, 128, 10)
    return segments

In [ ]:
new_train = combine_feature(x_new_train)
test = combine_feature(x_test)
new_train.shape, test.shape

**Convolution neural network** is widely used in Image processing and feature extraction known as 2D CNN in which kernel moves in two dimentions or 3 dimensions and learns important spatial features. In this question, given dataset contains time-series data in which 128 different sensor measurements are given. In 1D CNN network, kernel can move in only 1 direction that is in the direction of the time/samples. 


Input to the CNN model is, matrix of size (128, 10) where 128 is the sample size and 10 is the number of features. First 1D CNN maps all sensors values to 128 values and this it does for all 128 time samples and creates matrix of 128,128.

Next 1D CNN layer uses 64 filters for 64 different sampled features and with kernel size (3,1). This layer learns how single feature depends on each other over time. 

TOTAL NUMBER OF PARAMETERS USED : 73,753
AVG TRAINING ACCURACY : 0.85
AVG VALIDATION ACCURACY : 0.75

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, MaxPooling1D
from keras.layers import Conv1D

In [ ]:
def my_cnn():
    model = Sequential()
    
    model.add(Conv1D(filters = 128, kernel_size = 1, input_shape = (128,10), activation = 'relu'))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.1))

    model.add(Conv1D(filters = 64, kernel_size = 3, activation = 'relu'))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.1))

    model.add(Conv1D(filters = 32,kernel_size = 5, activation = 'relu'))
    model.add(Conv1D(filters = 32,kernel_size = 5, activation = 'relu'))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.1))

    model.add(Conv1D(filters = 16,kernel_size = 5, activation = 'relu'))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Dropout(0.1))
     
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Dropout(0.1))
    model.add(Dense(9, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


    return model

In [ ]:
model = my_cnn()
filepath = "skynet.hdf5"
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=10), 
                 keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)]
model.summary()

In [ ]:
model.fit(new_train, new_target, batch_size = 100, epochs = 200, callbacks=callbacks_list, validation_split = 0.1, shuffle = True, verbose = 1)

In [ ]:
prediction = model.predict(test)
prediction

**TESTING AGAINST DIFFERENT MACHINE LEARNING MODELS (LINEAR REGRESSION, SVM, KNeighbourclassifier) :**

In the main dataset, below function merges all the columns for the same measurement and find the values according to the operation. Like, it merges all the column and finds the maximum value out of all the values. Some of the useful functions have been used to extract feature information for different sensor data. 


In [ ]:
def SSC(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1 
    xn_i1 = x[0:len(x)-2]  # xn-1
    ans = np.heaviside((xn-xn_i1)*(xn-xn_i2),0)
    return sum(ans[1:])

def SRAV(x):    
    SRA = sum(np.sqrt(abs(x)))
    return np.power(SRA/len(x),2)

def _kurtosis(x):
    return kurtosis(x)

def feature_transformation(data):
    data_feature = pd.DataFrame()
    
    for col in data.columns:
        if col in ['row_id','series_id','measurement_number']:
            continue   
        data_feature[col + '_skew'] = data.groupby(['series_id'])[col].skew()
        data_feature[col + '_mad'] = data.groupby(['series_id'])[col].mad()
        data_feature[col + '_q70'] = data.groupby(['series_id'])[col].quantile(0.7)
        data_feature[col + '_q90'] = data.groupby(['series_id'])[col].quantile(0.9)
        data_feature[col + '_SSC'] = data.groupby(['series_id'])[col].apply(SSC) 
        data_feature[col + '_SRAV'] = data.groupby(['series_id'])[col].apply(SRAV)
        data_feature[col + '_mean'] = data.groupby(['series_id'])[col].mean()
        data_feature[col + '_median'] = data.groupby(['series_id'])[col].median()
        data_feature[col + '_max'] = data.groupby(['series_id'])[col].max()
        data_feature[col + '_min'] = data.groupby(['series_id'])[col].min()
        data_feature[col + '_std'] = data.groupby(['series_id'])[col].std()
        data_feature[col + '_range'] = data_feature[col + '_max'] - data_feature[col + '_min']
        data_feature[col + '_maxtoMin'] = data_feature[col + '_max'] / data_feature[col + '_min']
    return data_feature

train_feature = feature_transformation(x_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew

train_feature.fillna(0,inplace=True)
train_feature.replace(-np.inf,0,inplace=True)
train_feature.replace(-np.inf,0,inplace=True)

#predicted = np.zeros((test_feature.shape[0],9))
measured= np.zeros((train_feature.shape[0]))
score = 0
import gc
gc.enable()

y_train['surface'] = le.fit_transform(y_train['surface'])

**Linear regression**
Knowing the relationship between dependent variable and independent variable, linear regression can help in finding the value of the label which depends on the sensors features. Linear regression tries to find the parameters in such a way that it minimises the mean square error of the model. It uses hyper dimensional linear equation Y = \theta*X + b where X is the matrix of the feature. At the end, it predicts the values and tries to minimises the MSE and updates the parameter values.

AVG ACCURACY FOR 100 FOLDS : 0.2

In [ ]:
folds = StratifiedKFold(n_splits=100, shuffle=True, random_state=1234)

from sklearn.linear_model import LinearRegression
def linearRegression(measured, score, train_feature, y_train):
    for times, (t_idx, v_idx) in enumerate(folds.split(train_feature.values, y_train['surface'].values)):
        model = LinearRegression().fit(train_feature.iloc[t_idx], y_train['surface'][t_idx])
        measured[v_idx] = model.predict(train_feature.iloc[v_idx])
        score += model.score(train_feature.iloc[v_idx], y_train['surface'][v_idx])
        print("Fold: {} score: {}".format(times,model.score(train_feature.iloc[v_idx],y_train['surface'][v_idx])))    
        gc.collect()

print("Linear regression")
linearRegression(measured, score, train_feature, y_train)



**KNearestNeighbour Algorithm**

A data is assinged to class group to which it is most closed in terms of euclidean distance. 

AVERAGE ACCURACY : 0.6

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
def KNeighborsClassifier_():
    score = 0
    for times, (trn_idx, val_idx) in enumerate(folds.split(train_feature.values, y_train['surface'].values)):
        model = KNeighborsClassifier().fit(train_feature.iloc[trn_idx], y_train['surface'][trn_idx])
        measured[val_idx] = model.predict(train_feature.iloc[val_idx])
        score += model.score(train_feature.iloc[val_idx], y_train['surface'][val_idx])
        print("Fold: {} score: {}".format(times,model.score(train_feature.iloc[val_idx],y_train['surface'][val_idx])))    
        gc.collect()



print("KNeighboursclassifiers")
KNeighborsClassifier_(),

**Support Vector Machines**
After observing in linear regression that the data seems like linearly inseperable, support vector machine's polynomial and rbf kernels can be used to effectively classify multiclass. It seperates hyper plane with the maximum margin. 

AVERAGE TRAINING ACCURACY : 0.4

In [ ]:
from sklearn.svm import SVC
def svm_model(train_feature,  y_train):
    for times, (trn_idx, val_idx) in enumerate(folds.split(train_feature.values,y_train['surface'].values)):
        model = SVC(kernel= 'rbf', C = 1)
        model.fit(train_feature.iloc[trn_idx],y_train['surface'][trn_idx])
        measured[val_idx] = model.predict(train_feature.iloc[val_idx])
        print("Fold: {} score: {}".format(times,model.score(train_feature.iloc[val_idx],y_train['surface'][val_idx])))
        gc.collect()

print("Support vector machine model")
svm_model(train_feature, y_train)

Trained CNN model is able to predict correct label given the data with the accuracy of 75% for the given test dataset in which 20 samples are fed as testing parameters. 

In [ ]:
pred_y = prediction.argmax(axis = 1)
pred_class = le.inverse_transform(pred_y)
num_correct = 0

for i in range(y_test.shape[0]) :
    if pred_class[i] == y_test['surface'][i] :
        num_correct += 1
        
print("Score on Testing Data =", num_correct / y_test.shape[0])

In [ ]:
answer = pd.DataFrame(pred_class, columns = ['Surface'])
answer.to_csv('submission_final.csv', index=False)

In [ ]:
answer